In [3]:
%pip install azure-ai-formrecognizer
%pip install azure-storage-blob

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 15.6 MB/s eta 0:00:00
  Using cached cryptography-41.0.1-cp37-abi3-manylinux_2_28_x86_64.whl (4.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 25.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import os

endpoint = "https://fr-2i5ckl6yk5ina.cognitiveservices.azure.com/"
key = "2b4d5f2e615947458d608e8d4bef0e71"

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])

data = [
    "https://aikazukidcsstorage.blob.core.windows.net/source/20230502_to01.pdf",
]

from azure.storage.blob import BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string("DefaultEndpointsProtocol=https;AccountName=aikazukidcsstorage;AccountKey=JbeTh/jyeme/ksBNxQemNqc/xqfhj3emrqUu/5nWVDxOt7cIHpioleEgV4cKsNQAfsNF3HXTGqT2+AStukScxw==;EndpointSuffix=core.windows.net")
container_client = blob_service_client.get_container_client("source")
blob_list = container_client.list_blobs()

def analyze_read():
    for blob in blob_list:
        # sample document
        formUrl = f"https://aikazukidcsstorage.blob.core.windows.net/source/{blob.name}"
        filename = os.path.splitext(os.path.basename(formUrl))[0]

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )
        
        poller = document_analysis_client.begin_analyze_document_from_url(
                "prebuilt-read", formUrl)
        result = poller.result()

        with open(f"output/{filename}.txt", "w") as file:
            file.write(result.content)

    
    # for idx, style in enumerate(result.styles):
    #     print(
    #         "Document contains {} content".format(
    #             "handwritten" if style.is_handwritten else "no handwritten"
    #         )
    #     )

    # for page in result.pages:
    #     print("----Analyzing Read from page #{}----".format(page.page_number))
    #     print(
    #         "Page has width: {} and height: {}, measured with unit: {}".format(
    #             page.width, page.height, page.unit
    #         )
    #     )

    #     for line_idx, line in enumerate(page.lines):
    #         print(
    #             "...Line # {} has text content '{}' within bounding box '{}'".format(
    #                 line_idx,
    #                 line.content,
    #                 format_bounding_box(line.polygon),
    #             )
    #         )

    #     for word in page.words:
    #         print(
    #             "...Word '{}' has a confidence of {}".format(
    #                 word.content, word.confidence
    #             )
    #         )

    # print("----------------------------------------")


if __name__ == "__main__":
    analyze_read()
